<a href="https://colab.research.google.com/github/dibya-pati/CIFAR-10/blob/master/cifar_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Feb 22 08:11:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [0]:
%tensorflow_version 2.x
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
from collections import namedtuple
%tensorflow_version

TensorFlow 2.x selected.
Currently selected TF version: 2.x
Available versions:
* 1.x
* 2.x


In [0]:
# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()

	return trainX, trainY, testX, testY

In [0]:
@tf.function
def CeLU(x):
  alpha = tf.constant(0.075, dtype=tf.float32)
  #return tf.cond(tf.math.greater_equal(x, 0), x, alpha*(tf.math.exp(x/alpha) - 1))
  return tf.keras.activations.elu(x, alpha=0.075)

In [0]:
# define cnn model
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(Conv2D(64, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.2))
	model.add(Conv2D(64, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same'))
	model.add(BatchNormalization())
	model.add(Conv2D(128, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same', dilation_rate=2))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.5))
	model.add(Conv2D(128, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same', dilation_rate=2))
	model.add(BatchNormalization())
	model.add(Conv2D(192, (3, 3), activation=CeLU, kernel_initializer='he_uniform',\
	                 padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.5))
	model.add(tf.keras.layers.GlobalAveragePooling2D())
	#model.add(Flatten())
	#model.add(Dense(128, activation=CeLU, kernel_initializer='he_uniform'))
	#model.add(BatchNormalization())
	#model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	return model


In [0]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()

In [0]:
def augtrain(X,y):
  X = tf.cast(X, tf.float16)
  X = tf.pad(X, [[6,6], [6,6], [0,0]], mode="SYMMETRIC")
  X = tf.image.random_crop(X, [32, 32, 3])
  X = tf.image.random_flip_left_right(X)
  X = tf.image.random_brightness(X, 0.2)  

  return X, y

def augtest(X,y):
  return tf.cast(X, tf.float16), y

In [0]:
def tfdatagen(X, y, batchsize=256, epochs=50, train=True):
  cif_mean = \
        np.array([0.4913997551666284, 0.48215855929893703, 0.4465309133731618])
  cif_std = \
        np.array([0.24703225141799082, 0.24348516474564, 0.26158783926049628])

  X = X.astype(np.float32)/255.0
  y = y.astype(np.float32)

  X = (X - cif_mean)/cif_std

  y = to_categorical(y)
  
  if train:
    y = np.where(y==1, 0.8, 0.2/9)
    dataset = tf.data.Dataset.from_tensor_slices((X,y)).repeat(epochs).\
              shuffle(buffer_size=10*batchsize).\
              map(augtrain, num_parallel_calls=tf.data.experimental.AUTOTUNE).\
              batch(batchsize).prefetch(tf.data.experimental.AUTOTUNE)
  else:
    dataset = tf.data.Dataset.from_tensor_slices((X,y)).repeat(epochs).\
              shuffle(buffer_size=10*batchsize).\
              map(augtest, num_parallel_calls=tf.data.experimental.AUTOTUNE).\
              batch(batchsize).prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

In [0]:
class PiecewiseLinear(namedtuple('PiecewiseLinear', ('knots', 'vals'))):
    def __call__(self, t):
        return np.interp([t], self.knots, self.vals)[0]


def scheduler(epoch):
  lr_schedule = PiecewiseLinear([0, 15, 20, 50], [0.0001, 0.004, 0.001, 0.0001])
  return lr_schedule(epoch)

def schedulert(epoch):
  baselr = 0.004
  if epoch < 12:
    return baselr
  else:
    return max(baselr * tf.math.exp(0.05 * (12 - epoch)), 0.00001)


In [0]:
# run the test harness for evaluating a model
def run_test_harness():
  trainX, trainY, testX, testY = load_dataset()
  # idx = np.arange(trainX.shape[0])
  # np.random.shuffle(idx)
  # splitperc = 1.0
  # trainValSplitIdx = int(splitperc*trainX.shape[0])
  batchsize = 256
  epochs = 60
  traingen = tfdatagen(trainX,trainY,batchsize=batchsize, epochs=epochs,\
                       train=True)
  #valgen = tfdatagen(trainX[idx[trainValSplitIdx:]], \
  #                   trainY[idx[trainValSplitIdx:]], \
  #                   batchsize=batchsize, epochs=epochs)
  testgen = tfdatagen(testX, testY, batchsize=batchsize, epochs=epochs,\
                      train=False)

  model = define_model()
  print(model.summary())
  opt = Adam(lr=0.004)
  loss = tf.keras.losses.CategoricalCrossentropy()
  model.compile(optimizer=opt, loss=loss,\
                metrics=['accuracy'])

  steps = int(trainX.shape[0] / batchsize)
  #steps_valid = int((1-splitperc)*trainX.shape[0] / batchsize)
  steps_test = int(testX.shape[0]/batchsize)

  callback = tf.keras.callbacks.LearningRateScheduler(schedulert, verbose=1)
  #callback = []
  history = model.fit(traingen, steps_per_epoch=steps, epochs=epochs,\
                      callbacks=[callback], validation_data=testgen, \
                      validation_steps=steps_test, verbose=1)
  
  _, acc = model.evaluate(testgen, verbose=1)
  print('> %.3f' % (acc * 100.0))
  summarize_diagnostics(history)

In [0]:
if __name__=='__main__':
  # entry point, run the test harness
  run_test_harness()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_156 (Conv2D)          (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_156 (Bat (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_157 (Conv2D)          (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_157 (Bat (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_78 (Dropout)         (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 16, 16, 64)      